### 1、第一个问题的input的设置

In [ ]:
input([prompt])
参数说明-prompt: 提示信息

### 2、第二个问题time.time()是什么意思

In [ ]:
返回当前时间的时间戳（1970纪元后经过的浮点秒数）。
http://www.runoob.com/python/att-time-time.html

### 3、如何将输出的dtype格式信息分行展示

In [ ]:
在‘User Type’的输出中，所展示的格式是pandas Series。
如何将其使用合适的方式展示出来？

### 4、出现缺失值怎么办？

In [ ]:
在‘Gender’和‘User Type’的项目下有很多NaN缺失值，在统计时如何处理？

## 试运行除虫发现问题

### 5、while 循环处的问题

invalid syntax 原来是while和else都忘了冒号：

In [ ]:
#另外试运行以下真的很傻
while city != "chicago" or city !="new york" or city != "washington":
    #这行代码是要把使用者逼上绝路啊
    #or 改成 and 

### 6、未定义，数据调用不过来

查看错误记录name 'city' is not defined，不在列表中的问题很诡异，应该是def get_filters()和def load_data(city, month, day)两个函数没有很好的衔接起来。

1，如果说输入的最终结果return city, month, day 是一个元组的话，是不是单独调用city出现了问题呢？似乎不像，因为在def load_data(city, month, day)里同时调用了三个数据。和之前成功练习但区别就在于这里是load_data(city, month, day)，之前是df = load_data('chicago', 'march', 'friday')

2,有没有可能是数据类型的问题？查询之前的案例df = pd.read_csv(filename)其中直接用字符串就可以读取，所以可能并不是数据类型问题。
将get_filters()函数的代码保存为test.py,发现返回了字符串数据。但为什么仍然会说城市没有定义呢？

3,有没有可能是几个csv文件没有一起导入？出现了调用缺失？将city直接换成'chicago'，发现调用并没有问题。

4，一步步在test文件中还原函数调用，直到city, month, day = get_filters() print(city) print(CITY_DATA[city])都没有任何问题，说明可能是def load_data(city, month, day)出了问题。

比如输出测试这三个值：
print(CITY_DATA[city])  ---chicago.csv
df = load_data(city, month, day) --None
df1 = pd.read_csv(chicago.csv) ---not defined
df2 = pd.read_csv(str(chicago.csv)) ---not defined
df3 = pd.read_csv('chicago.csc')  ---right!

莫非是自己对pd.read_csv（）方法有什么误解？

（重读操作文档：http://pandas.pydata.org/pandas-docs/stable/io.html 再补充一下必要的参数，水真的好深：https://blog.csdn.net/u013066730/article/details/58634061）

然后还是发现必须是字符串格式，所以强行加了'str'   df = pd.read_csv(str(CITY_DATA[city]))
然后在test.py中运行成功，但是回到主文件再次失败。

再次看报错，是71行，第二次出现df = pd.read_csv(str(CITY_DATA[city]))的时候
time_stats(df)
  File "bikeshare.py", line 71, in time_stats
仔细一想，原来是自己没有弄清楚函数之间值传递关系。

def main():
    while True:
        city, month, day = get_filters()

        df = load_data(city, month, day)

        time_stats(df)
        station_stats(df)
        trip_duration_stats(df)
        user_stats(df)

        restart = input('\nWould you like to restart? Enter yes or no.\n')
        if restart.lower() != 'yes':
            break
            
也就是说，在df的值出现之后，后面的函数都直接调用了df的数据，这个数据中并没有city的值，而且这时再使用pd.read_csv就显得很多此一举了。            

## 审阅之后的问题

### 7、容错率比较低

代码没有考虑到输入大小写的问题。审阅教练建议全部小写化处理
city = input('Chicago, new york city and washington, Which city would you like to explore? ').lower()

### 8、数据集清点不认真

华盛顿数据集中缺少 Gender 和 Birth Year 列，如果对华盛顿的数据集做这两列的统计计算，就会报错 KeyError。

尝试用try except

各种异常说明及排除方法 http://www.runoob.com/python/python-exceptions.html

In [ ]:
try:
            user_stats(df)
        except KeyError:
            print("Sorry,the Gender data of this city is missed.")

### 9、冗余太多

In [ ]:
df['Start Time'] =  pd.to_datetime(df['Start Time'])
#这样的操作进行了数次
#关于这里的新建列的工作是有所冗余的，因为这个数据集在load_data函数中就已经进行这个列的新建工作了，这里就无需再次进行了。
#但删去后，一开始会出现这个报错
 TypeError: 'NoneType' object is not subscriptable
        #原因应该是数值没有传导过来,但在第一步筛选print结果就是none，所以应该是修改第一部分但结果
        
       #原来是删掉了 return df

### 10、函数参数值输出

In [ ]:
#经过审阅老师的指点，把第一部分城市、月份、周几的输入检验打包成函数
def input_check(v_list, input_message, error_message):
        while input_message not in v_list:
            print(error_message)
            new1 = input('try again\n')
            input_message=input_message.replace(input_message,new1)
        return input_message

In [ ]:
##但最大的问题是，出错后更新的参数值，无法返回到原先的数据中。

#猜测1、可能是while 循环的判断问题，当变量满足值之后直接跳走，没有输出新的值。在while之前或者输入之后增加v的值还是没有起色。
    #由于函数要赋给city，month，和day三部分的值，所以也不好直接指定。
    #最终借助while else 把输出限定在正确值
    
#猜测2、也有可能是从input_check到get_filters()传导结构的问题。
    #将原来简单的return v,city=v.替换为
    city = input_check(CITY_DATA.keys(),"Would you like to see data for Chicago,New York,or Washington?\n","please choose from the Chicago,New York and Washington.")
    #出现ValueError: None is not in list的问题
    def input_check(v_list, input_message, error_message):
    v = input(input_message).lower()
    while v not in v_list:
        print(error_message)
        v = input("try again.\n").lower()
    else:
        return v
    #结合while出现满足值之后就跳走的特征，把否定条件前置。
    
##猜测3、是否与不可变对象，函数改变不了参数的值有关。
    #所以尝试用replace替换。http://www.cnblogs.com/monkey-moon/p/9347505.html
    #https://blog.csdn.net/together_cz/article/details/70172083

    #当然，后来换成下面的直接赋值一点问题也没有。
v = input("try again.\n").lower()

这部分攻关最大的收获就是加深了对于函数和循环的理解。比如所有有规律的代码其实都有成为函数的潜质。而结合循环、输入的函数其实可以非常高效。之前对于break、continue等循环结构不是很清楚，这次也了解了大体的使用情景。